In [1]:
!pip install pymongo

In [ ]:
from pymongo import MongoClient
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv(override=True)

uri = os.getenv("MONGO_URI")
db_name = os.getenv("MONGO_DB_NAME")
collection_name = os.getenv("COLLECTION_NAME")

client = MongoClient(uri)
db = client[db_name]
collection = db[collection_name]

# MongoDB에서 데이터 불러오기
data = list(collection.find())

# DataFrame으로 변환
df = pd.DataFrame(data)

In [115]:
df.head()

,_id,investSessionId,chapterId,childId,turn,riskLevel,currentPoint,beforeValue,currentValue,initialValue,numberOfShares,income,transactionType,plusClick,minusClick,startedAt,endedAt,_class
0,b'oI\xae\x87\xbdcU\xc1\xe5}\xa5\x8d5\xba\\\xb2',b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',1111,"b'33""""\x11\x11\x11\xc1UUUUUUDD'",1,고위험 고수익,900,0,100,100,5,0,BUY,5,0,2025-06-10 13:55:14,2025-06-10 13:55:18,com.popoworld.backend.invest.entity.InvestHistory
1,b'4J\x1f\xc6\xaf\xfdw[F\xf1\x97\x80\x8dS\x8e\x99',b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',1111,"b'33""""\x11\x11\x11\xc1UUUUUUDD'",1,균형형,900,0,100,100,3,0,BUY,3,0,2025-06-10 13:55:14,2025-06-10 13:55:18,com.popoworld.backend.invest.entity.InvestHistory
2,b'\x81J\xf4\xad\xc3_\x01\x95!\x95\x07\x1b(\xf9...,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',1111,"b'33""""\x11\x11\x11\xc1UUUUUUDD'",1,장기 안정형,900,0,100,100,3,0,BUY,3,0,2025-06-10 13:55:14,2025-06-10 13:55:18,com.popoworld.backend.invest.entity.InvestHistory
3,b'\xbbO\x1b\xb4j<\x81\x9f>\x08\x1b\x07z\xeb@\xb5',b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',1111,"b'33""""\x11\x11\x11\xc1UUUUUUDD'",2,균형형,1670,100,110,100,1,20,SELL,0,2,2025-06-10 13:55:18,2025-06-10 13:55:22,com.popoworld.backend.invest.entity.InvestHistory
4,b':BtFH\xba\r%w.\xcfo1A\x0f\x92',b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',1111,"b'33""""\x11\x11\x11\xc1UUUUUUDD'",2,고위험 고수익,1670,100,90,100,1,-40,SELL,0,4,2025-06-10 13:55:18,2025-06-10 13:55:22,com.popoworld.backend.invest.entity.InvestHistory


In [116]:
df.groupby(["investSessionId","chapterId","turn"])[["startedAt","endedAt"]].value_counts()

investSessionId                                    chapterId  turn  startedAt            endedAt            
b'BK8G\xda\x03\x1e\xe3\xf4\x16o\x97I\x88S\x99'     1111       1     2025-06-10 13:55:56  2025-06-10 13:56:08    3
                                                              2     2025-06-10 13:56:08  2025-06-10 13:56:19    3
                                                              3     2025-06-10 13:56:19  2025-06-10 13:56:22    3
                                                              4     2025-06-10 13:56:22  2025-06-10 13:56:23    3
                                                              5     2025-06-10 13:56:23  2025-06-10 13:56:24    3
                                                              6     2025-06-10 13:56:24  2025-06-10 13:56:25    3
                                                              7     2025-06-10 13:56:25  2025-06-10 13:56:26    3
b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80'  1111       1     2025-06-10 13:55:14  2025

# 데이터 전처리

In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   _id              42 non-null     object        
 1   investSessionId  42 non-null     object        
 2   chapterId        42 non-null     object        
 3   childId          42 non-null     object        
 4   turn             42 non-null     int64         
 5   riskLevel        42 non-null     object        
 6   currentPoint     42 non-null     int64         
 7   beforeValue      42 non-null     int64         
 8   currentValue     42 non-null     int64         
 9   initialValue     42 non-null     int64         
 10  numberOfShares   42 non-null     int64         
 11  income           42 non-null     int64         
 12  transactionType  42 non-null     object        
 13  plusClick        42 non-null     int64         
 14  minusClick       42 non-null     int64      

In [118]:
df["chapterId"].value_counts()

chapterId
1111    42
Name: count, dtype: int64

In [119]:
# transactionType 라벨 인코딩
df['transactionType_encoded'], uniques = pd.factorize(df['transactionType'])
df[["transactionType_encoded","transactionType"]].value_counts()

transactionType_encoded  transactionType
2                        KEEP               24
0                        BUY                14
1                        SELL                4
Name: count, dtype: int64

## news_tag 컬럼 추가

In [120]:
import random

# 'all', 'high', 'mid' 중 하나를 무작위로 선택하는 함수
def assign_random_category():
    return random.choice(['all', 'high', 'mid'])

# 'news_tag'라는 새로운 컬럼을 추가할 준비
df['newsTag'] = None

# 'investSessionId', 'chapterId', 'turn' 기준으로 그룹화
# 각 그룹에 대해 고유한 값을 할당
for name, group in df.groupby(['investSessionId', 'chapterId', 'turn']):
    # 현재 그룹에 할당할 무작위 값 선택
    assigned_value = assign_random_category()
    
    # 해당 그룹의 모든 행에 동일한 값 할당
    df.loc[group.index, 'newsTag'] = assigned_value

In [121]:
df.groupby(['investSessionId', 'chapterId', 'turn'])["newsTag"].value_counts()

investSessionId                                    chapterId  turn  newsTag
b'BK8G\xda\x03\x1e\xe3\xf4\x16o\x97I\x88S\x99'     1111       1     all        3
                                                              2     high       3
                                                              3     mid        3
                                                              4     high       3
                                                              5     high       3
                                                              6     high       3
                                                              7     mid        3
b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80'  1111       1     high       3
                                                              2     mid        3
                                                              3     mid        3
                                                              4     all        3
                                 

# buy_ratio_df
: 고/중/저위험 종목 구매비율
- ["transactionType"]=="BUY"

In [122]:
import pandas as pd
import numpy as np

In [ ]:
# 구매 데이터 생성
transaction_df = df[["investSessionId","turn","riskLevel","currentPoint","transactionType","numberOfShares"]].copy()
buy_df = transaction_df[transaction_df["transactionType"]=="BUY"]

# 구매 수량 컬럼 생성
buy_df["turn"]

,investSessionId,turn,riskLevel,currentPoint,transactionType,numberOfShares
0,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',1,고위험 고수익,900,BUY,5
1,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',1,균형형,900,BUY,3
2,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',1,장기 안정형,900,BUY,3
6,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',3,장기 안정형,280,BUY,5
7,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',3,고위험 고수익,280,BUY,8
8,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',3,균형형,280,BUY,5
9,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',4,고위험 고수익,20,BUY,9
10,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',4,장기 안정형,20,BUY,7
21,b'BK8G\xda\x03\x1e\xe3\xf4\x16o\x97I\x88S\x99',1,균형형,200,BUY,5
22,b'BK8G\xda\x03\x1e\xe3\xf4\x16o\x97I\x88S\x99',1,고위험 고수익,200,BUY,8


In [ ]:
# 구매 데이터 생성
transaction_df = df[["investSessionId","riskLevel","currentPoint","transactionType","numberOfShares"]]
buy_df = transaction_df[transaction_df["transactionType"]=="BUY"]

# investSessionId별 구매 개수(전체 구매 횟수)
total_buy = buy_df.groupby("investSessionId")["numberOfShares"].sum().reset_index().rename(columns={"numberOfShares":"total_buy"})

# 병합 DataFrame
buy_ratio_df = total_buy


### 1️⃣ 종목별 구매 개수 ###

# 반복문을 위한 riskLevel 리스트와 해당 컬럼명 매핑 딕셔너리
risk_levels = {
    "고위험 고수익": "high",
    "균형형": "mid",
    "장기 안정형": "low"
}

# 각 riskLevel 별로 구매 수량 합하기
for risk_level, col_name in risk_levels.items():
    
    # riskLevel 필터링
    filtered_buy = buy_df[buy_df["riskLevel"] == risk_level]
    
    # investSessionId 별 구매 수량 합
    risk_level_summary = filtered_buy.groupby("investSessionId")["numberOfShares"].sum().reset_index().rename(columns={"numberOfShares":f"{col_name}_buy"})
    
    # 데이터 병합
    buy_ratio_df = pd.merge(buy_ratio_df, risk_level_summary, on="investSessionId", how="left")

# nan 값은 0으로 대체
buy_ratio_df.fillna(0, inplace=True)


### 2️⃣ 종목별 구매 비율 ###

# 매핑 컬럼명
ratio_columns = {
    "high_buy": "high_buy_ratio",
    "mid_buy": "mid_buy_ratio",
    "low_buy": "low_buy_ratio"
}

# 비율 구하기
for original_col, col_name in ratio_columns.items():
    buy_ratio_df[col_name] = buy_ratio_df[original_col] / buy_ratio_df["total_buy"]

buy_ratio_df

,investSessionId,total_buy,high_buy,mid_buy,low_buy,high_buy_ratio,mid_buy_ratio,low_buy_ratio
0,b'BK8G\xda\x03\x1e\xe3\xf4\x16o\x97I\x88S\x99',37,8,11,18,0.216216,0.297297,0.486486
1,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',45,22,8,15,0.488889,0.177778,0.333333


# sell_ratio_df
: 고/중/저위험 종목 판매 비율
- ["transactionType"]=="SELL"

In [95]:
# 판매 데이터 생성
transaction_df = df[["investSessionId","riskLevel","currentPoint","transactionType","numberOfShares"]]
sell_df = transaction_df[transaction_df["transactionType"]=="SELL"]

# investSessionId별 판매 개수(전체 판매 횟수)
total_sell = sell_df.groupby("investSessionId")["numberOfShares"].sum().reset_index().rename(columns={"numberOfShares":"total_sell"})

# 병합 DataFrame
sell_ratio_df = total_sell


### 1️⃣ 종목별 판매 개수 ###

# 반복문을 위한 riskLevel 리스트와 해당 컬럼명 매핑 딕셔너리
risk_levels = {
    "고위험 고수익": "high",
    "균형형": "mid",
    "장기 안정형": "low"
}

# 각 riskLevel 별로 판매 수량 합하기
for risk_level, col_name in risk_levels.items():
    
    # riskLevel 필터링
    filtered_sell = sell_df[sell_df["riskLevel"] == risk_level]
    
    # investSessionId 별 판매 수량 합
    risk_level_summary = filtered_sell.groupby("investSessionId")["numberOfShares"].sum().reset_index().rename(columns={"numberOfShares":f"{col_name}_sell"})
    
    # 데이터 병합
    sell_ratio_df = pd.merge(sell_ratio_df, risk_level_summary, on="investSessionId", how="left")

# nan 값은 0으로 대체
sell_ratio_df.fillna(0, inplace=True)


### 2️⃣ 종목별 판매매 비율 ###

# 매핑 컬럼명
ratio_columns = {
    "high_sell": "high_sell_ratio",
    "mid_sell": "mid_sell_ratio",
    "low_sell": "low_sell_ratio"
}

# 비율 구하기
for original_col, col_name in ratio_columns.items():
    sell_ratio_df[col_name] = sell_ratio_df[original_col] / sell_ratio_df["total_sell"]

sell_ratio_df

,investSessionId,total_sell,high_sell,mid_sell,low_sell,high_sell_ratio,mid_sell_ratio,low_sell_ratio
0,b'BK8G\xda\x03\x1e\xe3\xf4\x16o\x97I\x88S\x99',6,6,0.0,0.0,1.000000,0.000000,0.000000
1,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',3,1,1.0,1.0,0.333333,0.333333,0.333333


# tag_df
:평균 tag뉴스 발생 턴 체류시간

In [96]:
# 턴별 체류시간
tag_turn_df = df[["investSessionId","turn","newsTag","startedAt","endedAt"]].drop_duplicates()

tag_turn_df = tag_turn_df[tag_turn_df["newsTag"] != "all"]
tag_turn_df["turnDuration"] = tag_turn_df["endedAt"] - tag_turn_df["startedAt"]

# 게임별 평균 뉴스턴 체류시간
tag_df = tag_turn_df.groupby("investSessionId")["turnDuration"].mean().reset_index().rename(columns={"turnDuration":"tagTrunDuraion"})
tag_df

,investSessionId,tagTrunDuraion
0,b'BK8G\xda\x03\x1e\xe3\xf4\x16o\x97I\x88S\x99',0 days 00:00:04.285714285
1,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',0 days 00:00:04.333333333


# risk_taking
: 위험 감수율

In [111]:
risk_taking = df[["investSessionId","turn","newsTag","riskLevel","numberOfShares", "transactionType", "beforeValue","currentValue","income"]].copy()
risk_taking

,investSessionId,turn,newsTag,riskLevel,numberOfShares,transactionType,beforeValue,currentValue,income
0,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',1,high,고위험 고수익,5,BUY,0,100,0
1,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',1,high,균형형,3,BUY,0,100,0
2,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',1,high,장기 안정형,3,BUY,0,100,0
3,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',2,high,균형형,1,SELL,100,110,20
4,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',2,high,고위험 고수익,1,SELL,100,90,-40
5,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',2,high,장기 안정형,1,SELL,100,95,-10
6,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',3,all,장기 안정형,5,BUY,95,105,0
7,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',3,all,고위험 고수익,8,BUY,90,70,0
8,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',3,all,균형형,5,BUY,110,120,0
9,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',4,all,고위험 고수익,9,BUY,70,60,0


In [ ]:
risk_taking = df[["investSessionId","turn","newsTag","riskLevel","numberOfShares", "transactionType", "income"]].copy()

# 조건 리스트 생성
conditions = [
    (df["newsTag"]=="high") & (df["riskLevel"]=="고위험 고수익"),
    (df["newsTag"]=="mid") & (df["riskLevel"]=="균형형"),
    (df["newsTag"]=="all")
]

# 각 조건별 입력할 위험 발생률률
risk = [
    0.5,
    0.3,
    0
]

# riskRatio 값 입력
risk_taking["riskRatio"] = np.nan
risk_taking["riskRatio"] = np.select(conditions, risk, default=np.nan)

# 턴별 위험 감수율 계산 (tag 종목 구매 수량 * riskRatio)
risk_taking["risktaking"] = risk_taking["numberOfShares"] * risk_taking["riskRatio"]

# 게임별 평균 위험 감수율? 총 위험감수율?
risk_taking = risk_taking.groupby("investSessionId")["risktaking"].mean().reset_index()
risk_taking

,investSessionId,risktaking
0,b'BK8G\xda\x03\x1e\xe3\xf4\x16o\x97I\x88S\x99',2.271429
1,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',0.300000


# bet_win
: 배팅 성공률

In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   _id                      42 non-null     object        
 1   investSessionId          42 non-null     object        
 2   chapterId                42 non-null     object        
 3   childId                  42 non-null     object        
 4   turn                     42 non-null     int64         
 5   riskLevel                42 non-null     object        
 6   currentPoint             42 non-null     int64         
 7   beforeValue              42 non-null     int64         
 8   currentValue             42 non-null     int64         
 9   initialValue             42 non-null     int64         
 10  numberOfShares           42 non-null     int64         
 11  income                   42 non-null     int64         
 12  transactionType          42 non-null  

In [ ]:
bet_win = df[["investSessionId","turn","newsTag","riskLevel","beforeValue","currentValue"]].copy()
bet_win

,investSessionId,turn,newsTag,riskLevel,beforeValue,currentValue
0,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',1,high,고위험 고수익,0,100
1,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',1,high,균형형,0,100
2,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',1,high,장기 안정형,0,100
3,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',2,high,균형형,100,110
4,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',2,high,고위험 고수익,100,90
5,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',2,high,장기 안정형,100,95
6,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',3,all,장기 안정형,95,105
7,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',3,all,고위험 고수익,90,70
8,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',3,all,균형형,110,120
9,b'LIzf\x1c\x0e\x13\x1c\xa7\x01\xee\xc8\x88G2\x80',4,all,고위험 고수익,70,60
